## 1. Module Load
- Modules
    - ``myDB`` : DataBase Handling Module
    - ``prec_crawling`` : Crawler Handling Module
    - ``prec_preprocessing`` : Preprocessing Module
    - ``model`` : Text Embeddings Model Handling Module

In [1]:
from modules import myDB, prec_crawling, prec_preprocessing, model

## 2. Package Load
- Packages
    - ``base64`` : Vector를 Byte 형태로 변환하기 위한 패키지
    - ``pandas`` : Data를 Handling하기위한 패키지
    - ``numpy`` : Byte를 Vector로 변환하기 위한 패키지

In [2]:
import base64
import pandas as pd
import numpy as np

## 3. DataBase Connect
- Info
    - host : localhost
    - user : root
    - password : 1234
    - DB Name : cases

In [3]:
# 1. myDB 객체 생성
db = myDB.CaseLawDB(host="localhost", user="root", password="1234", db="cases")
# 2. Database Connect
db.connection()

DB Connection Success


## 4. Modules Object declaration

In [4]:
# 1. Crawler module 객체 선언
crawler_module = prec_crawling.PrecedentCrawl()
# 2. 전처리 객체 선언
preprocessing_module = prec_preprocessing.PrecedentPrcs()
# 3. 임베딩 모델 객체 선언
embedding_module = model.PrecSearchEngine()

## 4. Function definition by function

### 1) Define First Crawling Function 
- 설명
    - 처음 크롤링을 할 때 사용하는 함수
    - 비어있는 데이터 베이스에 크롤링한 판례 데이터와 임베딩 데이터를 저장한다.
- 동작순서
    1. 판례 크롤링
    2. 판례 전처리
    3. 판례 임베딩
    4. 임베딩 Vetor -> Byte로 변환
    5. DB에 저장

In [5]:
def first_crawling():
    # 1. Crawling
    prec_dict = crawler_module.start_crawling()
    # 2. Convert Dictionary to DataFrame
    prec_df = crawler_module.dict_to_df(prec_dict)
    # 3. Preprocessing
    clean_prec_df = preprocessing_module.run(prec_df=prec_df)
    # 4. Embedding
    embedding_df = pd.DataFrame(columns=['판례일련번호', '임베딩'])
    embedding_df['판례일련번호'] = clean_prec_df['판례일련번호']
    embedding_df['임베딩'] = clean_prec_df['판례내용'].apply(embedding_module.embedding_text)
    # 5. Convert Vector to Byte
    embedding_df['임베딩'] = embedding_df['임베딩'].apply(base64.b64encode)
    embedding_df['임베딩'] = embedding_df['임베딩'].apply(base64.b64decode)
    # 6. Insert to DataBase
    db.insert(clean_prec_df, 'case_info')
    db.insert(embedding_df, 'case_embedding')

### 2) Define Update Crawling Function
- 설명
    - 데이터 베이스의 판례와 비교하여 추가된 판례만 크롤링하여 업데이트한다.
    - 기준은 판례의 판례일련번호를 기준으로 한다.
- 동작순서
    1. 데이터베이스에서 판례 데이터를 로드
    2. 판례 데이터를 선고일자 순으로 정렬
    3. 해당 선고일자를 기준으로 크롤링(추가된 판례만 반영된다)
    4. 추가된 판례를 전처리
    5. 추가된 판례를 임베딩
    6. 임베딩 Vector -> Byte 변환
    7. DataBase에 저장

In [10]:
def update_crawling():
    # 1. Load origin data from DataBase
    non_update_df = db.read("SELECT * FROM case_info")
    # 2. Sorted data by non-ascending
    non_update_df = non_update_df.sort_values("선고일자", ascending=False).reset_index(drop=True)
    # 3. Get the most recent case serial number
    most_recent_serial_nums = [str(non_update_df['판례일련번호'][i]) for i in range(10)]
    # 4. Update
    update_dict = crawler_module.update_crawling(recent_serial_nums=most_recent_serial_nums)
    ## Checking Update
    if update_dict:
        pass
    else:   
        print("Aready Update")
        return
    # 5. Convert Dictionary to DataFrame
    update_df = crawler_module.dict_to_df(update_dict)
    # 6. Preprocessing
    clean_update_df = preprocessing_module.run(update_df)
    # 7. Embedding
    embedding_df = pd.DataFrame(columns=['판례일련번호', '임베딩'])
    embedding_df['판례일련번호'] = clean_update_df['판례일련번호']
    embedding_df['임베딩'] = clean_update_df['판례내용'].apply(embedding_module.embedding_text)
    # 8. Convert Vector to Byte
    embedding_df['임베딩'] = embedding_df['임베딩'].apply(base64.b64encode)
    embedding_df['임베딩'] = embedding_df['임베딩'].apply(base64.b64decode)
    # 9. Insert to DataBase
    db.insert(clean_update_df, 'case_info')
    db.insert(embedding_df, 'case_embedding')

### 3) Define Search Function
- 설명
    - Nodejs 서버로부터 전달받은 데이터를 토대로 판례를 검색하는 함수
    - 데이터는 JSON 형식으로 전달받는다.
    - Output도 JSON 형식으로 전달한다.
- 동작순서
    1. JSON -> DataFrame 변환
    2. 사용자 데이터 분리(사건종류, 사용자 입력)
    3. 사용자 데이터 기반 쿼리문 생성
    4. 생성한 쿼리문으로 DataBase에서 판례 로드
    5. 생성한 쿼리문으로 DataBases에서 판례 임베딩 로드
    6. 판례와 판례 임베딩 병합 (key = 판례일련번호)
    7. 임베딩 값 Byte -> Vector 변환
    8. 사용자 입력과 판례 간의 유사도 계산
    9. 유사도를 %로 변환 (소수점 첫째자리까지)
    10. ['판례일련번호', '유사도']의 열만 분리
    11. 분리한 Sub DataFrame을 JSON으로 변환 후 리턴

In [14]:
def search(input_json):
    # 1. Convert Json to DataFrame
    input_df = pd.read_json(input_json)
    # 2. Split Types and Content
    input_types = input_df['type'].to_list()
    input_content = input_df['content'][0]
    # 3. Create Query
    query_condition = ""
    if len(input_types) == 1:
        query = f"사건종류명 = '{input_types[0]}'"
    else:
        for i in range(1, len(input_types)):
            query_condition += f"or 사건종류명 = '{input_types[i]}'"
        query = f"사건종류명 = '{input_types[0]}'" + query_condition
    # 4. Load Selected Data from DataBase
    load_prec_df = db.read(f"SELECT * FROM case_info WHERE {query}")
    # 5. Get Embedding vector from DataBase
    load_embedding_df = db.read(f"SELECT * FROM case_embedding WHERE 판례일련번호 IN (SELECT 판례일련번호 FROM case_info WHERE {query})")
    # 6. Merge by 판례일련번호
    merge_df = pd.merge(left=load_prec_df, right=load_embedding_df, how='inner', on='판례일련번호')
    # 7. Convert Byte to Vector
    merge_df['임베딩'] = merge_df['임베딩'].apply(lambda x : np.frombuffer(x, dtype=np.float32))
    # 8. Search
    searched_list = embedding_module.run(input_content, len(merge_df), merge_df)
    # 9. Concat similarity to DataFrame
    similarity = [sim for _, sim in searched_list]
    merge_df["유사도"] = similarity
    merge_df = merge_df.drop(['임베딩'], axis=1).reset_index(drop=True)
    # 10. Similarity decimal conversion
    merge_df['유사도'] = merge_df['유사도'].apply(lambda x : round(x, 3) * 100)
    # 11. Convert to JSON : ['판례일련번호', '유사도']
    sub_df = merge_df[['판례일련번호', '유사도']]
    sub_json = sub_df.to_json(orient='values')
    
    return sub_json

---
## 5. Module Testing

In [8]:
import time

### 1) first_crawling Test 

In [9]:
start = time.time()
first_crawling()
print(f"operating time : {round(time.time() - start, 2)}(sec)")

총 판례 수	:	85000
총 페이지 수	:	4250


  0%|          | 0/4250 [00:00<?, ?it/s]

insert time : 43.92(sec)
insert time : 26.15(sec)
operating time : 15219.35(sec)


### 2) update_crawling Test

In [11]:
start = time.time()
update_crawling()
print(f"operating time : {round(time.time() - start, 2)}(sec)")

read time : 9.26(sec)
총 판례 수	:	85000
총 페이지 수	:	4250


  0%|          | 0/4250 [00:00<?, ?it/s]

Aready Update
operating time : 9.64(sec)


### 3) search Test
- 사용자의 입력은 Nodejs 서버에서 받았다고 가정 (JSON)

In [12]:
json_string = '''{
    "type" : ["민사", "형사"],
    "content" : "사건 내용은 1년넘게 사귄 남자친구가 있었습니다 그 남자친구가 바람피고 저한테 걸려서 헤어지는과정에 말다툼을했었는데, 저한테 마주치면 코뼈부쉰다,팬다 어쩐다 협박을 했었어요. 몇일뒤 지나가는길에 그 남자친구를 마주쳤습니다 남자친구와 그의 친구들 총 5명이 있었는데 저한테 얘기를 하자고하길래 친구들있으니 저쪽가서 이야기하자 라고 하고 자리를 옮겼어요. (옮겼지만 그 친구들은 다시 남자친구와 제가 있는쪽으로 와서 구경했고요) 자리 옮기고 나서 얘기를 하려고 하니 얘기는커녕 그때 무슨생각으로 자기한테 욕을했냐 라면서 따지기 시작했고 일방적으로 욕을 먹기 시작했습니다. 얘기중에 손으로 제 팔 내려치길래 저는 그애가 분노조절도 못하고 충동적인 행동을 하는것을 잘 알기에 진정해라,이래서 너한테 좋을거없다 라고 수차례 얘기했음에도 자기 화를 이기지못해 저를 폭행했습니다. 머리를 손으로 내려치고 발로 팔을 차서 핸드폰 날라가서 망가지고, 제가 걔를 쳐다보니 뭘보냐면서 안면상그대로 손으로 맞았습니다 그러다 같이있던 제 친구가 112에 신고를하고 경찰분들 오기를 기다렸어요 그러다 사건접수하고 전 119타고 대학병원응급실가서 검사와 약처방받고 집으로 갔습니다. 경찰서까지 넘어가서 조사받고 단순 폭행으로 검찰 넘어가는중이에요",
}'''

In [15]:
start = time.time()
output_json = search(json_string)
print(f"operating time : {round(time.time() - start, 2)}(sec)")

read time : 9.4(sec)
read time : 7.71(sec)
operating time : 17.71(sec)


In [16]:
output_json

'[[64437,28.5],[64438,24.6],[64440,19.5],[64441,23.3],[64442,24.3],[64443,22.6],[64444,23.5],[64445,22.7],[64446,19.2],[64447,23.2],[64448,17.2],[64449,22.0],[64450,22.1],[64451,22.0],[64452,21.2],[64453,20.0],[64454,23.7],[64455,26.7],[64456,23.0],[64457,23.6],[64470,22.2],[64472,26.0],[64473,24.4],[64474,23.6],[64475,25.3],[64476,27.2],[64477,24.9],[64478,28.6],[64493,20.4],[64494,23.4],[64495,28.4],[64496,29.2],[64497,20.4],[64498,22.8],[64499,29.1],[64500,30.7],[64501,20.9],[64502,21.9],[64503,26.5],[64504,25.9],[64515,37.9],[64516,33.0],[64517,32.9],[64518,27.5],[64519,33.5],[64520,29.1],[64521,35.0],[64522,27.2],[64523,30.1],[64524,26.2],[64525,18.5],[64526,23.3],[64527,26.2],[64528,25.0],[64529,23.5],[64539,26.9],[64541,19.8],[64542,21.1],[64543,24.4],[64544,24.3],[64545,26.4],[64546,24.0],[64547,21.5],[64555,28.1],[64556,19.3],[64557,24.0],[64558,20.2],[64559,17.0],[64560,28.0],[64561,30.1],[64562,22.4],[64564,24.2],[64565,26.2],[64566,20.4],[64568,31.2],[64569,27.8],[64570,27.